## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-24-21-47-33 +0000,cbc,"Amid buyback blowback, Carney faces his first ...",https://www.cbc.ca/news/politics/carney-ananda...
1,2025-09-24-21-46-00 +0000,wsj,Opinion | A New Start for Trump on Ukraine?,https://www.wsj.com/opinion/donald-trump-russi...
2,2025-09-24-21-44-00 +0000,wsj,Opinion | When the Left Censored Speech,https://www.wsj.com/opinion/google-alphabet-yo...
3,2025-09-24-21-41-50 +0000,wapo,Live updates: Suspect identified in Dallas ICE...,https://www.washingtonpost.com/nation/2025/09/...
4,2025-09-24-21-40-06 +0000,nyt,"U.N. General Assembly Updates: China, for Firs...",https://www.nytimes.com/live/2025/09/24/world/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2309/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
10,trump,54
8,new,16
475,kimmel,15
28,china,15
21,ice,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
287,2025-09-24-01-43-00 +0000,wsj,When Trump announced changes to the H-1B visa ...,https://www.wsj.com/politics/policy/air-india-...,118
162,2025-09-24-14-05-20 +0000,nypost,Trump threatens to sue ABC over Jimmy Kimmel’s...,https://nypost.com/2025/09/24/us-news/trump-th...,106
263,2025-09-24-05-04-00 +0000,wsj,Kimmel Makes Emotional Return to ABC With Trum...,https://www.wsj.com/business/media/jimmy-kimme...,104
233,2025-09-24-09-30-00 +0000,wsj,Stephen Miran set out his case for lower inter...,https://www.wsj.com/economy/central-banking/st...,99
8,2025-09-24-21-39-41 +0000,wapo,President Donald Trump wrote on TruthSocial th...,https://www.washingtonpost.com,97


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
287,118,2025-09-24-01-43-00 +0000,wsj,When Trump announced changes to the H-1B visa ...,https://www.wsj.com/politics/policy/air-india-...
143,70,2025-09-24-15-46-02 +0000,bbc,Jimmy Kimmel calls out 'anti-American' threats...,https://www.bbc.com/news/articles/c9dx6dlelw8o...
295,63,2025-09-24-00-42-57 +0000,cbc,Carney says he will meet China's president 'at...,https://www.cbc.ca/news/politics/carney-china-...
16,51,2025-09-24-21-35-00 +0000,wsj,Shooting at Dallas ICE Facility Leaves One Det...,https://www.wsj.com/us-news/dallas-texas-ice-o...
153,43,2025-09-24-15-00-55 +0000,nypost,New home sales surge over 20% to 3-year high a...,https://nypost.com/2025/09/24/business/new-hom...
151,42,2025-09-24-15-06-31 +0000,bbc,Israeli forces near Gaza City centre as Hamas ...,https://www.bbc.com/news/articles/c4gk9dreldxo...
207,41,2025-09-24-10-24-08 +0000,cbc,Typhoon Ragasa makes landfall in China after k...,https://www.cbc.ca/news/world/typhoon-ragasa-c...
103,34,2025-09-24-18-22-17 +0000,bbc,Russia will expand aggression beyond Ukraine i...,https://www.bbc.com/news/articles/c5yg921rjrko...
68,32,2025-09-24-19-50-36 +0000,nypost,Career criminal accused of shoving off-duty NY...,https://nypost.com/2025/09/24/us-news/homeless...
222,32,2025-09-24-10-00-00 +0000,nypost,Vivek Ramaswamy tells ‘Pod Force One’ US may h...,https://nypost.com/2025/09/24/us-news/vivek-ra...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
